In [9]:
import sys
sys.path.append("../../")
import os
import datetime
import pandas as pd
import numpy as np
from example_lorenz import get_lorenz_data
from sindy_utils import library_size
from training import train_network
import torch

# Generate data

In [10]:
# # generate training, validation, testing data
# noise_strength = 1e-6
# training_data = get_lorenz_data(100, noise_strength=noise_strength) #1024
# validation_data = get_lorenz_data(10, noise_strength=noise_strength) #20



In [11]:
# Function to convert data to tensors and move to GPU
def convert_to_tensors(data_dict, device):
    for key, value in data_dict.items():
        if value is not None:
            data_dict[key] = torch.tensor(value).float().to(device)
    return data_dict

# Generate training, validation data
noise_strength = 1e-6
training_data = get_lorenz_data(1024, noise_strength=noise_strength) # 1024
validation_data = get_lorenz_data(20, noise_strength=noise_strength) # 20

# Specify the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Convert training and validation data to tensors and move to GPU
training_data = convert_to_tensors(training_data, device)
validation_data = convert_to_tensors(validation_data, device)

# If 'ddx' is required and should be set to None
training_data['ddx'] = None
validation_data['ddx'] = None

# Set up model and training parameters

In [12]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 3
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 1
params['coefficient_mask'] = torch.ones((params['library_dim'], params['latent_dim']))
#print('params[coefficient_mask]:', params['coefficient_mask'].shape)
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.0
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
print(params['epoch_size'])
params['batch_size'] = 1024 #1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 1

# training time cutoffs
params['max_epochs'] = 5001 #5001
params['refinement_epochs'] = 1001 #1001

25000


# Run training experiments

In [13]:

num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = torch.ones((params['library_dim'], params['latent_dim'])).to("cuda:0")

    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    results_dict = train_network(training_data, validation_data, params)
    # print(type(df))
    # df = df.concat({**results_dict, **params}, ignore_index=True)
    df = pd.concat([df, pd.DataFrame([{**results_dict, **params}])], ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
Sequential thresholding is enabled
val_data None
TRAINING


c:\Users\zhaox\OneDrive - Delft University of Technology\桌面\Q4\reproductory\sindy_reproduction\sindy_reproduction\example\lorenz\../..\training.py:226: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'x': torch.tensor(data['x'][idxs], dtype=torch.float32).to(device),
c:\Users\zhaox\OneDrive - Delft University of Technology\桌面\Q4\reproductory\sindy_reproduction\sindy_reproduction\example\lorenz\../..\training.py:227: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'dx': torch.tensor(data['dx'][idxs], dtype=torch.float32).to(device)
c:\Users\zhaox\OneDrive - Delft University of Technology\桌面\Q4\reproductory\sindy_reproduction\sindy_reproduction\example\lorenz\../..\training.py:234: UserWarning

Epoch 0
   Training Losses:
      Total Loss: 0.10385086387395859
      decoder Loss: 0.10238063335418701
      sindy_z Loss: 55.16654586791992
      sindy_x Loss: 14.604644775390625
      sindy_regularization Loss: 0.9771175384521484
   Validation Losses:
      Total Loss: 0.09781479090452194
      decoder Loss: 0.09678245335817337
      sindy_z Loss: 51.758419036865234
      sindy_x Loss: 10.225693702697754
      sindy_regularization Loss: 0.9771175384521484
Decoder Loss Ratio: 0.527618, Decoder SINDy Loss Ratio: 1.000746
Epoch 1
   Training Losses:
      Total Loss: 0.0988842323422432
      decoder Loss: 0.09741421788930893
      sindy_z Loss: 60.327178955078125
      sindy_x Loss: 14.604752540588379
      sindy_regularization Loss: 0.9538058042526245
   Validation Losses:
      Total Loss: 0.09159348160028458
      decoder Loss: 0.09056136012077332
      sindy_z Loss: 52.64815139770508
      sindy_x Loss: 10.225862503051758
      sindy_regularization Loss: 0.9538058042526245
Decode